In [ ]:
import os
import json
import tensorflow as tf
import IPython
import functools
import matplotlib.pyplot as plt
import numpy as np
import io # iostreams
from PIL import Image # image processing
from tqdm import tqdm # ranges are displayed 

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition --unzip
#!kaggle datasets download -d maysee/mushrooms-classification-common-genuss-images --unzip

flowers-recognition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
os.listdir(os.getcwd())

['.deepnote',
 'flowers_values',
 'init.ipynb',
 'mushrooms',
 'flowers-recognition.zip',
 'flowers',
 'notebook.ipynb',
 'requirements.txt']

In [ ]:
print("Flower types: \n", os.listdir("flowers"))
print("Mushroom types: \n", os.listdir("mushrooms"))

Flower types: 
 ['daisy', 'dandelion', 'rose', 'tulip', 'sunflower']
Mushroom types: 
 ['Suillus', 'Agaricus', 'Cortinarius', 'Hygrocybe', 'Lactarius', 'Amanita', 'Russula', 'Boletus', 'Entoloma']


In [ ]:
def nd_index(i, *args):
    vals = []
    for v in range(len(args)):
        vals.append(i%args[v])
        i = i // args[v]
    return vals

In [ ]:
numbers = []
for i in tqdm(range(1, 20)):
    for j in tqdm(range(1, 20)):
        numbers.append(i*20+j)
    

100%|██████████| 19/19 [00:00<00:00, 136225.26it/s]

100%|██████████| 19/19 [00:00<00:00, 177883.43it/s]

100%|██████████| 19/19 [00:00<00:00, 134614.49it/s]

100%|██████████| 19/19 [00:00<00:00, 30828.54it/s]

100%|██████████| 19/19 [00:00<00:00, 184045.67it/s]

100%|██████████| 19/19 [00:00<00:00, 102563.42it/s]

100%|██████████| 19/19 [00:00<00:00, 158432.95it/s]

100%|██████████| 19/19 [00:00<00:00, 8302.96it/s]

100%|██████████| 19/19 [00:00<00:00, 157182.99it/s]

100%|██████████| 19/19 [00:00<00:00, 180706.98it/s]

100%|██████████| 19/19 [00:00<00:00, 235078.99it/s]

100%|██████████| 19/19 [00:00<00:00, 181944.69it/s]

100%|██████████| 19/19 [00:00<00:00, 171749.52it/s]

100%|██████████| 19/19 [00:00<00:00, 279620.27it/s]

100%|██████████| 19/19 [00:00<00:00, 279620.27it/s]

100%|██████████| 19/19 [00:00<00:00, 249036.80it/s]

100%|██████████| 19/19 [00:00<00:00, 190650.18it/s]

100%|██████████| 19/19 [00:00<00:00, 273854.90it/s]

100%|██████████| 19/19 [00:00<00:00, 332.13it/s]


In [ ]:
def mkdir (name):
    if not os.path.exists(name):
        os.mkdir(name)

In [ ]:
lens = {}
import os
for category in os.listdir("flowers"):
    lens.update({category: len(os.listdir("flowers" + "/" + category))})
print(lens)

{'daisy': 769, 'dandelion': 1055, 'rose': 784, 'tulip': 984, 'sunflower': 734}


In [ ]:
root = "flowers"
out_root = "flowers_values"

list_of_categories = os.listdir(root)

img_size = (120, 240)

# tensors are saved in this directory
mkdir(out_root)

# total number of iterations in two nested for loops
# for category in list_of_categories:
#     for image_index in len_category:
total = len(list_of_categories) * 1054
for total_i in tqdm(range(total)):

    if total_i < 3141:
        continue
    # it is like decimal to binary conversion: every total_i has a unique combination
    # of category_i and image_i
    category_i, image_i = nd_index(total_i, len(list_of_categories), 1054)

    category = list_of_categories[category_i]

    # a directory for the category itself
    mkdir(out_root + "/" + category)

    images_in_category = os.listdir(root+"/"+category)

    # this gives us the real length of images
    len_category = len(images_in_category)
    if (image_i < len_category):

        # image names are given as name.jpg
        image_name = images_in_category[image_i]

        # if the image format is not jpg then filter it 
        if (any([image_name.endswith(x) for x in [".jpg", ".jpeg"]])):

            # the json file name of respective image 
            new_image_name = ".".join(image_name.split(".")[:-1]) + ".json"

            # image is loaded
            image = Image.open(root+"/"+category+"/"+image_name)

            # images resized to the given size
            image = image.resize(img_size)

            # 3d list, RGB values in separate height and width
            new_tensors = tf.keras.preprocessing.image.img_to_array(image).tolist()

            # write the new tensors as a separate file
            with open(out_root + "/" + category + "/" + new_image_name, "w") as file:
                file.write(json.dumps(new_tensors, indent=4))

100%|██████████| 5270/5270 [04:01<00:00, 21.86it/s]


In [ ]:
#parameters of CNN
num_filters = 5

#make model
def cnn_classifier(num_outputs = len(list_of_categories)):
    Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
    BatchNormalization = tf.keras.layers.BatchNormalization
    Flatten = tf.keras.layers.Flatten
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')
    MaxPool2D = functools.partial(tf.keras.layers.MaxPool2D, pool_size=(2, 2), strides=None, padding="same")
    model = tf.keras.Sequential([    
        # size = (120, 240)
        
        Conv2D(filters=32, kernel_size=5,  strides=2),
        BatchNormalization(),
        MaxPool2D(),
        
        Conv2D(filters=2*num_filters, kernel_size=5,  strides=2),
        BatchNormalization(),
        MaxPool2D(),

        Conv2D(filters=4*num_filters, kernel_size=3,  strides=2),
        BatchNormalization(),
        MaxPool2D(),

        Conv2D(filters=6*num_filters, kernel_size=3,  strides=2),
        BatchNormalization(),

        Flatten(),
        Dense(512),
        Dense(num_outputs, activation="softmax"),
        
    ])
    return model

cnn_model = cnn_classifier()

In [ ]:
cnn_model.compile(optimizer="sgd", loss="""sparse or normal categorical_crossentropy""", metrics=["accuracy"])

In [ ]:
batch_size = 10
output = cnn_model(np.random.random(size = (batch_size, 120, 240, 3)))

In [ ]:
def predict (output):
    return [list_of_categories[np.argmax(x)] for x in output]

In [ ]:
predict(output)

['rose',
 'rose',
 'rose',
 'rose',
 'rose',
 'rose',
 'rose',
 'rose',
 'rose',
 'rose']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=647c4d53-d8d6-4b2a-b042-e3f4bdfbc4d0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>